In [181]:
import numpy as np
import pandas as pd
import time
import datetime
import sys
import os
from tqdm import tqdm
from functools import reduce
from Bio import SeqIO
import joblib
sys.path.append("../tools/")
import funclib

sys.path.append("../")
import benchmark_train as btrain
import benchmark_test as btest
import benchmark_common as bcommon
import config as cfg
import benchmark_evaluation as eva
import embedding_esm as esmebd

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from pandarallel import pandarallel #  import pandaralle
pandarallel.initialize() # init

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#read train test data
train = pd.read_feather(cfg.DATADIR+'task3/train.feather')
test = pd.read_feather(cfg.DATADIR+'task3/test.feather')
print('train size: {0}\ntest size: {1}'.format(len(train), len(test)))

train size: 222567
test size: 3304


In [10]:
train.iloc[0].index.values

array(['id', 'seq', 'ec_number'], dtype=object)

In [ ]:
funclib.split_ecdf_to_single_lines_pr_record

In [16]:
resDff = pd.DataFrame(columns=test.iloc[0].index.values)

In [116]:
input_df

,id,seq
0,t1,MAFLKRRDLISIKSLFLVLFLGLVSLSICEQEKREEENQEEDEENE...
1,t2,MAFLKKSLFLVLFLGKKKKKKKKKKKLVSLSICEKEKRQNEEDEDE...
2,t3,MARGGDTGCTGPSETSASMMMMMMMMMGAVAIAFPGLEGPPADAQY...
3,P69031,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...
4,P69019,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...
...,...,...
7099,A0A061ACU2,MTVPPLLKSCVVKLLLPAALLAAAIIRPSFLSIGYVLLALVSAVLP...
7100,Q8GYS8,MKIGVVLVLLTVFVVVMSSTSVSAQSDEDECLKETGQMQLNCFPYL...
7101,Q6NRV0,MPIRAYCTICSDFFDNARDVAAITCGHTFHQECLLQWFHSAPHRTC...
7102,C5DLH0,MGLSADLFVQKRSAASSLKQPKELGFYSKTQEGQFLVNDSSKLSYY...


In [112]:
print('step 1: loading data') 
input_df = funclib.load_fasta_to_table(cfg.DATADIR + 'test1.fasta') # test fasta
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER) #sprot db


step 1: loading data


In [145]:
find_data =input_df.merge(latest_sprot, on='seq', how='left')
exist_data= find_data[~find_data.name.isnull()].iloc[:,np.r_[0,2,1,12,7,9:12]].rename(columns={'id_x':'id','id_y':'id_uniprot'})
noExist_data = find_data[find_data.name.isnull()]
noExist_data.reset_index(drop=True, inplace=True)
noExist_data = noExist_data.iloc[:,np.r_[0,2,1,12,7,9:12]].rename(columns={'id_x':'id','id_y':'id_uniprot'})

In [146]:
exist_data

,id,id_uniprot,seq,seqlength,ec_number,date_integraged,date_sequence_update,date_annotation_update
3,P69031,P69031,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...,71.0,-,01-FEB-2005,17-JUN-2020,02-JUN-2021
4,P69019,P69019,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...,72.0,-,01-FEB-2005,17-JUN-2020,02-JUN-2021
5,Q5RF96,Q5RF96,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,169.0,3.4.-.-,30-AUG-2005,16-OCT-2019,02-JUN-2021
6,Q9UM00,Q9UM00,MPRKRKCDLRAVRVGLLLGGGGVYGSRFRFTFPGCRALSPWRVRVQ...,239.0,-,27-JUN-2006,10-APR-2019,02-JUN-2021
7,P0C250,P0C250,MGKLTILVLVAAVLLSTQVMVQGDGDQPADRNAVRRDDNPGGTRGR...,63.0,-,28-NOV-2006,10-OCT-2018,02-JUN-2021
...,...,...,...,...,...,...,...,...
7199,A0A061ACU2,A0A061ACU2,MTVPPLLKSCVVKLLLPAALLAAAIIRPSFLSIGYVLLALVSAVLP...,2442.0,-,02-DEC-2020,03-SEP-2014,02-JUN-2021
7200,Q8GYS8,Q8GYS8,MKIGVVLVLLTVFVVVMSSTSVSAQSDEDECLKETGQMQLNCFPYL...,116.0,-,02-DEC-2020,01-MAR-2003,02-JUN-2021
7201,Q6NRV0,Q6NRV0,MPIRAYCTICSDFFDNARDVAAITCGHTFHQECLLQWFHSAPHRTC...,464.0,2.3.2.27,02-DEC-2020,02-DEC-2020,02-JUN-2021
7202,C5DLH0,C5DLH0,MGLSADLFVQKRSAASSLKQPKELGFYSKTQEGQFLVNDSSKLSYY...,383.0,"3.6.1.-, 3.6.1.-",02-DEC-2020,28-JUL-2009,07-APR-2021


In [90]:
funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)

Write finished


In [163]:
# 1. 读入数据
print('step 1: loading data') 
input_df = funclib.load_fasta_to_table(cfg.DATADIR +'test1.fasta') # test fasta
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER) #sprot db

# 2. 查找数据
print('step 2: find existing data')
find_data =input_df.merge(latest_sprot, on='seq', how='left')
exist_data= find_data[~find_data.name.isnull()].iloc[:,np.r_[0,2,1,12,7,9:12]].rename(columns={'id_x':'id','id_y':'id_uniprot'})
noExist_data = find_data[find_data.name.isnull()]
noExist_data.reset_index(drop=True, inplace=True)
noExist_data = noExist_data.iloc[:,np.r_[0,2,1,12,7,9:12]].rename(columns={'id_x':'id','id_y':'id_uniprot'})

# 3. EMBedding
print('step 3: Embedding')
rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=noExist_data, model='esm1b_t33_650M_UR50S',seqthres=1022)

# 4. 获取序列比对结果
print('step 4: sequence alignment')
if os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB) == False:
    funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)
    cmd = r'diamond makedb --in {0} -d {1}'.format(cfg.FILE_BLAST_PRODUCTION_FASTA, cfg.FILE_BLAST_PRODUCTION_DB)
    os.system(cmd)

blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=noExist_data)
blast_res = blast_res[['id', 'sseqid']].merge(latest_sprot, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,2:14]]
blast_res = blast_res.iloc[:,np.r_[0,1,11,12,6,8:11]].rename(columns={'id_x':'id','id_y':'id_uniprot'})


step 1: loading data
step 2: find existing data
step 3: Embedding
Transferred model to GPU


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 26.06it/s]


step 4: sequence alignment
Write finished
diamond blastp -d /home/shizhenkun/codebase/DMLF/data/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


In [162]:
os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB)

True

In [179]:
rep32.iloc[:,1:]

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f1271,f1272,f1273,f1274,f1275,f1276,f1277,f1278,f1279,f1280
0,-6.804794,2.874747,-2.936860,9.810719,-6.819678,-17.022587,-13.554860,-0.945068,-21.338894,-9.093796,...,8.640766,0.553148,-5.229982,-5.618551,-20.637119,-1.331101,1.730169,1.479166,7.273944,-1.048626
1,0.807973,2.875237,0.469086,-3.014995,0.587386,-3.174599,-8.452457,-0.436728,-18.501244,6.350846,...,8.353586,3.589030,0.058293,-3.295337,-45.453663,-7.404720,8.713732,-2.032311,7.031470,-6.454871
2,7.572537,-5.488810,2.927005,2.439370,-0.462251,-8.748297,-10.529886,-4.403286,-3.329234,-6.083371,...,11.920851,-0.599013,-7.007697,2.774383,-25.021109,1.580775,2.467106,-1.006016,8.282257,3.572978
3,3.416424,-2.389026,2.426224,7.088617,-11.862194,-4.401493,0.032177,-2.899314,-6.869094,0.416968,...,9.655196,-1.627206,5.404442,0.844533,-28.978022,-2.528508,5.510448,0.752212,3.254661,7.857780
4,0.857835,3.731980,-2.733037,3.602181,-5.794148,-2.742023,-5.606553,-5.034950,-1.415696,-5.164866,...,9.676998,0.650701,-5.822467,-3.127125,-9.022912,-0.864405,1.988653,-0.474267,-0.413070,9.110717
5,-5.730790,4.211004,-3.075476,1.002254,-7.551456,-3.089291,-13.591138,2.205750,-9.584989,-9.263247,...,14.738030,0.385222,-12.335782,4.743570,3.566010,-5.412627,6.517208,-0.024651,-0.994412,7.483754
6,3.835431,-0.138807,-3.428080,1.010092,-3.972997,-3.186929,-8.622228,1.809379,-1.290898,-8.476718,...,17.268738,1.427671,-6.782789,6.861602,-9.555772,-3.494762,10.467276,4.030308,-1.440476,1.230301


In [190]:
model_isEnzyme = joblib.load(cfg.ISENZYME_MODEL)
pred_isEnzyme = pd.DataFrame()
pred_isEnzyme['id']=rep32.id
pred_isEnzyme['isEnzyme_pred'] = model_isEnzyme.predict(rep32.iloc[:,1:])

In [191]:
pred_isEnzyme

,id,isEnzyme_pred
0,t1,0
1,t2,0
2,t3,0
3,C0HLG3,1
4,U4PCM1,0
5,F2SH39,0
6,W6QRN8,1


In [189]:
isEnzyme_pred['id']=rep32.id

NameError: name 'isEnzyme_pred' is not defined

In [186]:
res_data

,id_x,id_y,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,t1,P69031,AUR52_RANAE,False,False,0,-,0,01-FEB-2005,17-JUN-2020,02-JUN-2021,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...,71
1,t2,P69019,AUR25_RANAE,False,False,0,-,0,01-FEB-2005,17-JUN-2020,02-JUN-2021,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...,72
2,t3,Q5RF96,SPCS1_PONAB,True,False,1,3.4.-.-,2,30-AUG-2005,16-OCT-2019,02-JUN-2021,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,169
3,C0HLG3,C0HLG3,RNAG_AGRAE,True,False,1,4.6.1.23,4,16-JAN-2019,10-FEB-2021,02-JUN-2021,MSESSTFTTAVVPEGEGVAPMAETVQYYNSYSDASIASCAFVDSGK...,156
4,U4PCM1,U4PCM1,WDR33_CAEEL,False,False,0,-,0,08-MAY-2019,07-APR-2021,02-JUN-2021,MNGGMMRGNQMPNVTLTIQPSTSSMQNSQPRIMNNHHHPHNRFQRE...,809
5,F2SH39,F2SH39,MFS1_TRIRC,False,False,0,-,0,13-NOV-2019,10-FEB-2021,10-FEB-2021,MTEKGADGADGLTKAKSNAVSEDYETVNHVTGLKLAVIVTGLCLSV...,536
6,W6QRN8,W6QRN8,MPADE_PENRF,True,False,1,1.-.-.-,1,26-FEB-2020,10-FEB-2021,02-JUN-2021,MDYLIIIRITAVAVVLYLTRYVCCLYLHLQDVPGPLFAKFTNLQRV...,852


In [ ]:
pred_s=model_s.predict(X_test)

In [177]:
aa

,id,id_uniprot,seq,seqlength,ec_number,date_integraged,date_sequence_update,date_annotation_update
3,P69031,P69031,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...,71.0,-,01-FEB-2005,17-JUN-2020,02-JUN-2021
4,P69019,P69019,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...,72.0,-,01-FEB-2005,17-JUN-2020,02-JUN-2021
5,Q5RF96,Q5RF96,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,169.0,3.4.-.-,30-AUG-2005,16-OCT-2019,02-JUN-2021
6,Q9UM00,Q9UM00,MPRKRKCDLRAVRVGLLLGGGGVYGSRFRFTFPGCRALSPWRVRVQ...,239.0,-,27-JUN-2006,10-APR-2019,02-JUN-2021
7,P0C250,P0C250,MGKLTILVLVAAVLLSTQVMVQGDGDQPADRNAVRRDDNPGGTRGR...,63.0,-,28-NOV-2006,10-OCT-2018,02-JUN-2021
...,...,...,...,...,...,...,...,...
2,t3,Q5RF96,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,169.0,3.4.-.-,30-AUG-2005,16-OCT-2019,02-JUN-2021
3,C0HLG3,C0HLG3,MSESSTFTTAVVPEGEGVAPMAETVQYYNSYSDASIASCAFVDSGK...,156.0,4.6.1.23,16-JAN-2019,10-FEB-2021,02-JUN-2021
4,U4PCM1,U4PCM1,MNGGMMRGNQMPNVTLTIQPSTSSMQNSQPRIMNNHHHPHNRFQRE...,809.0,-,08-MAY-2019,07-APR-2021,02-JUN-2021
5,F2SH39,F2SH39,MTEKGADGADGLTKAKSNAVSEDYETVNHVTGLKLAVIVTGLCLSV...,536.0,-,13-NOV-2019,10-FEB-2021,10-FEB-2021


In [172]:
len(set(aa.id))

7104

In [101]:
res_data = res_data[['id', 'sseqid']].merge(train, left_on='sseqid', right_on='id', how='left')[['id_x', 'isenzyme']]
res_data =res_data.rename(columns={'id_x':'id','isenzyme':'isenzyme_blast'})
res_data = test[['id','isenzyme']].merge(res_data, on='id', how='left')

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,Q6GZX4,001R_FRG3G,False,False,0,-,0,28-JUN-2011,19-JUL-2004,02-JUN-2021,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...,256
1,Q6GZX3,002L_FRG3G,False,False,0,-,0,28-JUN-2011,19-JUL-2004,02-JUN-2021,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...,320
2,Q197F8,002R_IIV3,False,False,0,-,0,16-JUN-2009,11-JUL-2006,02-JUN-2021,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...,458
3,Q197F7,003L_IIV3,False,False,0,-,0,16-JUN-2009,11-JUL-2006,12-AUG-2020,MYQAINPCPQSWYGSPQLEREIVCKMSGAPHYPNYYPVHPNALGGA...,156
4,Q6GZX2,003R_FRG3G,False,False,0,-,0,28-JUN-2011,19-JUL-2004,12-AUG-2020,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...,438
...,...,...,...,...,...,...,...,...,...,...,...,...
565249,Q6UY62,Z_SABVB,False,False,0,-,0,20-JAN-2009,05-JUL-2004,02-DEC-2020,MGNSKSKSKLSANQYEQQTVNSTKQVAILKRQAEPSLYGRHNCRCC...,100
565250,P08105,Z_SHEEP,False,False,0,-,0,01-AUG-1988,01-AUG-1988,11-DEC-2019,MSSSLEITSFYSFIWTPHIGPLLFGIGLWFSMFKEPSHFCPCQHPH...,79
565251,Q88470,Z_TACVF,False,False,0,-,0,06-DEC-2005,23-JAN-2007,02-JUN-2021,MGNCNRTQKPSSSSNNLEKPPQAAEFRRTAEPSLYGRYNCKCCWFA...,95
565252,A9JR22,Z_TAMVU,False,False,0,-,0,20-JAN-2009,05-FEB-2008,02-JUN-2021,MGLRYSKEVRDRHGDKDPEGRIPITQTMPQTLYGRYNCKSCWFANK...,95


In [196]:
rep32

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1271,f1272,f1273,f1274,f1275,f1276,f1277,f1278,f1279,f1280
0,t1,-6.804794,2.874747,-2.936860,9.810719,-6.819678,-17.022587,-13.554860,-0.945068,-21.338894,...,8.640766,0.553148,-5.229982,-5.618551,-20.637119,-1.331101,1.730169,1.479166,7.273944,-1.048626
1,t2,0.807973,2.875237,0.469086,-3.014995,0.587386,-3.174599,-8.452457,-0.436728,-18.501244,...,8.353586,3.589030,0.058293,-3.295337,-45.453663,-7.404720,8.713732,-2.032311,7.031470,-6.454871
2,t3,7.572537,-5.488810,2.927005,2.439370,-0.462251,-8.748297,-10.529886,-4.403286,-3.329234,...,11.920851,-0.599013,-7.007697,2.774383,-25.021109,1.580775,2.467106,-1.006016,8.282257,3.572978
3,C0HLG3,3.416424,-2.389026,2.426224,7.088617,-11.862194,-4.401493,0.032177,-2.899314,-6.869094,...,9.655196,-1.627206,5.404442,0.844533,-28.978022,-2.528508,5.510448,0.752212,3.254661,7.857780
4,U4PCM1,0.857835,3.731980,-2.733037,3.602181,-5.794148,-2.742023,-5.606553,-5.034950,-1.415696,...,9.676998,0.650701,-5.822467,-3.127125,-9.022912,-0.864405,1.988653,-0.474267,-0.413070,9.110717
5,F2SH39,-5.730790,4.211004,-3.075476,1.002254,-7.551456,-3.089291,-13.591138,2.205750,-9.584989,...,14.738030,0.385222,-12.335782,4.743570,3.566010,-5.412627,6.517208,-0.024651,-0.994412,7.483754
6,W6QRN8,3.835431,-0.138807,-3.428080,1.010092,-3.972997,-3.186929,-8.622228,1.809379,-1.290898,...,17.268738,1.427671,-6.782789,6.861602,-9.555772,-3.494762,10.467276,4.030308,-1.440476,1.230301


In [192]:
import production as pds

In [198]:
pds.predict_function_counts(test_data=rep32)

,id,pred_s,pred_m
0,t1,1,2
1,t2,1,2
2,t3,1,2
3,C0HLG3,1,2
4,U4PCM1,1,2
5,F2SH39,1,2
6,W6QRN8,1,2


In [194]:
rep32

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1271,f1272,f1273,f1274,f1275,f1276,f1277,f1278,f1279,f1280
0,t1,-6.804794,2.874747,-2.936860,9.810719,-6.819678,-17.022587,-13.554860,-0.945068,-21.338894,...,8.640766,0.553148,-5.229982,-5.618551,-20.637119,-1.331101,1.730169,1.479166,7.273944,-1.048626
1,t2,0.807973,2.875237,0.469086,-3.014995,0.587386,-3.174599,-8.452457,-0.436728,-18.501244,...,8.353586,3.589030,0.058293,-3.295337,-45.453663,-7.404720,8.713732,-2.032311,7.031470,-6.454871
2,t3,7.572537,-5.488810,2.927005,2.439370,-0.462251,-8.748297,-10.529886,-4.403286,-3.329234,...,11.920851,-0.599013,-7.007697,2.774383,-25.021109,1.580775,2.467106,-1.006016,8.282257,3.572978
3,C0HLG3,3.416424,-2.389026,2.426224,7.088617,-11.862194,-4.401493,0.032177,-2.899314,-6.869094,...,9.655196,-1.627206,5.404442,0.844533,-28.978022,-2.528508,5.510448,0.752212,3.254661,7.857780
4,U4PCM1,0.857835,3.731980,-2.733037,3.602181,-5.794148,-2.742023,-5.606553,-5.034950,-1.415696,...,9.676998,0.650701,-5.822467,-3.127125,-9.022912,-0.864405,1.988653,-0.474267,-0.413070,9.110717
5,F2SH39,-5.730790,4.211004,-3.075476,1.002254,-7.551456,-3.089291,-13.591138,2.205750,-9.584989,...,14.738030,0.385222,-12.335782,4.743570,3.566010,-5.412627,6.517208,-0.024651,-0.994412,7.483754
6,W6QRN8,3.835431,-0.138807,-3.428080,1.010092,-3.972997,-3.186929,-8.622228,1.809379,-1.290898,...,17.268738,1.427671,-6.782789,6.861602,-9.555772,-3.494762,10.467276,4.030308,-1.440476,1.230301
